In [1]:
from modules.environment_manager import get_notebook_path, enforce_venv, get_persistent_config_data

NOTEBOOK_NAME = "inversion_result_analysis.ipynb"

NOTEBOOK_PATH = get_notebook_path(NOTEBOOK_NAME)
enforce_venv(NOTEBOOK_PATH)
THREADS, MEMORY, HSI_CONFIG, PROJECT_FOLDER, OUTPUT_FOLDER = get_persistent_config_data(NOTEBOOK_PATH)

In [2]:
from PIL import Image
import numpy as np
import pandas as pd

PROCESSED_FOLDER = PROJECT_FOLDER / "data" / "processed"

hsi_name_0 = input("Please Enter: hog_island_2019_0 hsi name in hsi_config.json")
hsi_name_1 = input("Please Enter: hog_island_2019_1 hsi name in hsi_config.json")
hsi_name_0 = hsi_name_0 if hsi_name_0 else "uas_hsi_small"
hsi_name_1 = hsi_name_1 if hsi_name_1 else "uas_hsi_large"

In [3]:
def get_inv_df(hsi_name):
    inv_path = OUTPUT_FOLDER / f"{hsi_name}_inv_res.npy"
    inv_res = np.load(inv_path)

    if inv_res.shape[0] == 3705:
        hog_island_id = 0
    elif inv_res.shape[0] == 5321:
        hog_island_id = 1
    else:
        raise RuntimeError("Inversion dimensions do not match any hog_island_2019 images.")

    success = inv_res[:, :, 0]
    n = inv_res[:, :, 1]
    cab = inv_res[:, :, 2]
    ccx = inv_res[:, :, 3]
    ewt = inv_res[:, :, 4]
    lma = inv_res[:, :, 5]
    lai = inv_res[:, :, 6]
    psoil = inv_res[:, :, 7]
    rsoil = inv_res[:, :, 8]
    mask = inv_res[:, :, 9]

    df = pd.DataFrame(columns=["vp_names", "img_idx", "pix_y", "pix_x", "n", "cab", "ccx", "ewt", "lma", "lai", "psoil", "rsoil"])

    vp_lookup = np.array(
        Image.open(PROCESSED_FOLDER / f"vp_lookup_hog_island_2019_{hog_island_id}.png").convert("L"), dtype=np.uint8
    )

    df_idx = 0
    for i in range(vp_lookup.shape[0]):
        for j in range(vp_lookup.shape[1]):
            if success[i, j] > 0.99:
                vp_name = "vp_" + str(vp_lookup[i, j]).zfill(3)
                df.loc[df_idx] = [vp_name, hog_island_id, i, j, n[i, j], cab[i, j], ccx[i, j], ewt[i, j], lma[i,j], lai[i,j], psoil[i,j], rsoil[i,j]]
                df_idx += 1        
    return df

inv_df_0 = get_inv_df(hsi_name=hsi_name_0)
inv_df_1 = get_inv_df(hsi_name=hsi_name_1)
inv_df = pd.concat([inv_df_0, inv_df_1], ignore_index=True)
inv_df

,vp_names,img_idx,pix_y,pix_x,n,cab,ccx,ewt,lma,lai,psoil,rsoil
0,vp_030,0,1081,1510,1.095213,10.000000,8.267910,0.001941,0.034223,5.163403,0.719185,0.847475
1,vp_030,0,1082,1510,1.136846,10.000000,7.552181,0.001846,0.024383,4.081995,0.804409,0.737731
2,vp_030,0,1082,1511,1.140723,10.000000,7.461852,0.001905,0.032575,6.530915,0.919134,0.598911
3,vp_030,0,1082,1512,1.184393,10.000000,8.097132,0.001956,0.038317,5.216523,0.739252,0.733451
4,vp_030,0,1083,1509,1.000724,10.000000,7.000000,0.001890,0.027986,4.502490,0.866298,0.934582
...,...,...,...,...,...,...,...,...,...,...,...,...
3409,vp_080,1,2907,2839,1.033778,10.338265,7.000000,0.002000,0.076357,4.403463,0.704601,0.710270
3410,vp_080,1,2907,2840,1.121261,10.664826,7.515886,0.001982,0.048242,6.892568,0.709376,0.713260
3411,vp_080,1,2907,2841,1.028299,11.000689,7.000000,0.001978,0.069510,5.195986,0.741228,0.703044
3412,vp_080,1,2907,2842,1.028299,11.000689,7.000000,0.001978,0.069510,5.195986,0.741228,0.703044


In [4]:
print(inv_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   vp_names  3414 non-null   object 
 1   img_idx   3414 non-null   int64  
 2   pix_y     3414 non-null   int64  
 3   pix_x     3414 non-null   int64  
 4   n         3414 non-null   float64
 5   cab       3414 non-null   float64
 6   ccx       3414 non-null   float64
 7   ewt       3414 non-null   float64
 8   lma       3414 non-null   float64
 9   lai       3414 non-null   float64
 10  psoil     3414 non-null   float64
 11  rsoil     3414 non-null   float64
dtypes: float64(8), int64(3), object(1)
memory usage: 320.2+ KB
None


In [5]:
inv_df.to_csv(PROCESSED_FOLDER / "inv_res_hog_island_2019.csv")

In [6]:
inverted_vps = inv_df["vp_names"].unique()
inverted_vps.sort()
print(inverted_vps)
print("\nCount: ", len(inverted_vps))

['vp_003' 'vp_004' 'vp_005' 'vp_006' 'vp_008' 'vp_009' 'vp_010' 'vp_011'
 'vp_013' 'vp_014' 'vp_015' 'vp_016' 'vp_017' 'vp_018' 'vp_019' 'vp_020'
 'vp_021' 'vp_022' 'vp_023' 'vp_024' 'vp_025' 'vp_027' 'vp_028' 'vp_029'
 'vp_030' 'vp_037' 'vp_038' 'vp_039' 'vp_040' 'vp_046' 'vp_047' 'vp_050'
 'vp_051' 'vp_054' 'vp_062' 'vp_065' 'vp_071' 'vp_072' 'vp_073' 'vp_074'
 'vp_078' 'vp_080' 'vp_082' 'vp_085' 'vp_091']

Count:  45
